In [1]:
import os
import torch
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.tongyi import Tongyi
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain_openai import ChatOpenAI

# 加载嵌入模型
embedding_model = HuggingFaceEmbeddings(
    model_name="./bge-base-zh-v1.5",
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)

# 初始化 Chroma 客户端
vectorstore = Chroma(
    persist_directory="vectorstore",
    embedding_function=embedding_model,
)

# 创建检索器
retriever = vectorstore.as_retriever(
    search_type="similarity",  # 检索方式，similarity 或 mmr
    search_kwargs={"k": 3},
)


D:\software\anaconda3\envs\rag0319\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 检索与生成链条

# 1、获取大模型
load_dotenv()
TONGYI_API_KEY = os.getenv("TONGYI_API_KEY")
# 1、在线调用百炼平台的模型
llm = Tongyi(model="qwen-turbo", api_key=TONGYI_API_KEY)

# 2、通用的调用方式：在线调用其他平台、本地部署的模型（vllm、ollama）
# llm=ChatOpenAI(
#     model="qwen-turbo",# 如果vllm部署，必须与vllm启动时指定的model_server_name一致
#     api_key=TONGYI_API_KEY, # 如果vllm部署，内容随便填，但是这个参数不能为空
#     base_url="https://dashscope.aliyuncs.com/api/v1",#如果vllm部署，写vllm的地址，http://xxxxx:8000/v1
#     streaming=True,  #是否流式输出
#     extra_body={
#         "chat_template_kwargs": {"enable_thinking": False}  #可以指定关闭思考模式
#     },
# )


# 将检索到的文档中的 page_content 取出组合到一起
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# 2、Prompt 模板
prompt = PromptTemplate(
    input_variables=["context", "query"],
    template="""
你是一个专业的中文问答助手，擅长基于提供的资料回答用户问题。
请仅根据以下背景资料回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：{context}

问题：{query}

回答：""",
)

# 3、构建RAG 链条
# 在LCEL链条中，需要调用自己的函数，写成lambda表达式，就会自动转成Runable接口类型，不需要自己转
# 在LCEL链条中，需要调用自己的函数，中间有管道符，也会自动转成Runable接口类型，不需要自己转
rag_chain = (
        {"context": retriever | format_docs, "query": RunnablePassthrough()}
        | prompt
        | (lambda x: print(x.text, end="") or x) # or x 为了原封不动返回，不影响链条的传递
        | llm
        | StrOutputParser()  # 输出解析器，将输出解析为字符串
)

query = "不动产或者动产被占有人占有怎么办"
# query = "杀人犯法吗"
response = rag_chain.invoke(query)
print(response)



你是一个专业的中文问答助手，擅长基于提供的资料回答用户问题。
请仅根据以下背景资料回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：第九百九十八条　认定行为人承担侵害除生命权、身体权和健康权外的人格权的民事责任，应当考虑行为人和受害人的职业、影响范围、过错程度，以及行为的目的、方式、后果等因素。

第九百九十九条　为公共利益实施新闻报道、舆论监督等行为的，可以合理使用民事主体的姓名、名称、肖像、个人信息等；使用不合理侵害民事主体人格权的，应当依法承担民事责任。

第一千条　行为人因侵害人格权承担消除影响、恢复名誉、赔礼道歉等民事责任的，应当与行为的具体方式和造成的影响范围相当。

行为人拒不承担前款规定的民事责任的，人民法院可以采取在报刊、网络等媒体上发布公告或者公布生效裁判文书等方式执行，产生的费用由行为人负担。

第一千零一条　对自然人因婚姻家庭关系等产生的身份权利的保护，适用本法第一编、第五编和其他法律的相关规定；没有规定的，可以根据其性质参照适用本编人格权保护的有关规定。

第二章　生命权、身体权和健康权

第一千零二条　自然人享有生命权。自然人的生命安全和生命尊严受法律保护。任何组织或者个人不得侵害他人的生命权。

第一千一百八十条　因同一侵权行为造成多人死亡的，可以以相同数额确定死亡赔偿金。

第一千一百八十一条　被侵权人死亡的，其近亲属有权请求侵权人承担侵权责任。被侵权人为组织，该组织分立、合并的，承继权利的组织有权请求侵权人承担侵权责任。

被侵权人死亡的，支付被侵权人医疗费、丧葬费等合理费用的人有权请求侵权人赔偿费用，但是侵权人已经支付该费用的除外。

第一千一百八十二条　侵害他人人身权益造成财产损失的，按照被侵权人因此受到的损失或者侵权人因此获得的利益赔偿；被侵权人因此受到的损失以及侵权人因此获得的利益难以确定，被侵权人和侵权人就赔偿数额协商不一致，向人民法院提起诉讼的，由人民法院根据实际情况确定赔偿数额。

第一千一百八十三条　侵害自然人人身权益造成严重精神损害的，被侵权人有权请求精神损害赔偿。

因故意或者重大过失侵害自然人具有人身意义的特定物造成严重精神损害的，被侵权人有权请求精神损害赔偿。

第一千一百八十四条　侵害他人财产的，财产损失按照损失发生时的市场价格或者其他合理方式计算。

第一千一百八十五条　故意侵害他人知识产

In [5]:
# 带历史对话记录
load_dotenv()
TONGYI_API_KEY = os.getenv("TONGYI_API_KEY")

# 将检索到的文档中的 page_content 取出组合到一起
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt 模板
prompt = PromptTemplate(
    input_variables=["context", "history", "query"],
    template="""
你是一个专业的中文问答助手，擅长基于提供的资料回答问题。
请仅根据以下背景资料以及历史消息回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：{context}

历史消息：[{history}]

问题：{query}

回答：""",
)

# 大模型
llm = Tongyi(model="qwen-turbo", api_key=TONGYI_API_KEY)

# 历史消息
history = []

# 格式化历史消息
def format_history(history):
    # 只保留最近 3 轮对话记录
    max_epoch = 3
    if len(history) > 2 * max_epoch:
        history = history[-2 * max_epoch :]
    return "\n".join([f"{i['role']}：{i['content']}" for i in history])

# RAG 链条
rag_chain = (
    {
        "context": lambda x: format_docs(retriever.invoke(x["query"], k=3)),
        "history": lambda x: format_history(x["history"]),
        "query": lambda x: x["query"],
    }
    | prompt
    | (lambda x: print(x.text, end="") or x)
    | llm
    | StrOutputParser()  # 输出解析器，将输出解析为字符串
)

query_list = ["不动产或者动产被人占有怎么办", "那要是被损毁了呢"]
for query in query_list:
    print(f"===== 查询: {query} =====")
    response = rag_chain.invoke({"query": query, "history": history})
    print(response, end="\n\n")
    history.extend(
        [
            {"role": "用户", "content": query},
            {"role": "助手", "content": response},
        ]
    )


===== 查询: 不动产或者动产被人占有怎么办 =====

你是一个专业的中文问答助手，擅长基于提供的资料回答问题。
请仅根据以下背景资料以及历史消息回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：第五分编　占有

第二十章　占有

第四百五十八条　基于合同关系等产生的占有，有关不动产或者动产的使用、收益、违约责任等，按照合同约定；合同没有约定或者约定不明确的，依照有关法律规定。

第四百五十九条　占有人因使用占有的不动产或者动产，致使该不动产或者动产受到损害的，恶意占有人应当承担赔偿责任。

第四百六十条　不动产或者动产被占有人占有的，权利人可以请求返还原物及其孳息；但是，应当支付善意占有人因维护该不动产或者动产支出的必要费用。

第四百六十一条　占有的不动产或者动产毁损、灭失，该不动产或者动产的权利人请求赔偿的，占有人应当将因毁损、灭失取得的保险金、赔偿金或者补偿金等返还给权利人；权利人的损害未得到足够弥补的，恶意占有人还应当赔偿损失。

第四百六十二条　占有的不动产或者动产被侵占的，占有人有权请求返还原物；对妨害占有的行为，占有人有权请求排除妨害或者消除危险；因侵占或者妨害造成损害的，占有人有权依法请求损害赔偿。

占有人返还原物的请求权，自侵占发生之日起一年内未行使的，该请求权消灭。

第三编　合同

第一分编　通则

第一章　一般规定

第二百一十八条　权利人、利害关系人可以申请查询、复制不动产登记资料，登记机构应当提供。

第二百一十九条　利害关系人不得公开、非法使用权利人的不动产登记资料。

第二百二十条　权利人、利害关系人认为不动产登记簿记载的事项错误的，可以申请更正登记。不动产登记簿记载的权利人书面同意更正或者有证据证明登记确有错误的，登记机构应当予以更正。

不动产登记簿记载的权利人不同意更正的，利害关系人可以申请异议登记。登记机构予以异议登记，申请人自异议登记之日起十五日内不提起诉讼的，异议登记失效。异议登记不当，造成权利人损害的，权利人可以向申请人请求损害赔偿。

第二百二十一条　当事人签订买卖房屋的协议或者签订其他不动产物权的协议，为保障将来实现物权，按照约定可以向登记机构申请预告登记。预告登记后，未经预告登记的权利人同意，处分该不动产的，不发生物权效力。

预告登记后，债权消灭或者自能够进行不动产登记之日起九十日内未申

In [7]:
# 重述用户消息
load_dotenv()
TONGYI_API_KEY = os.getenv("TONGYI_API_KEY")

# 将检索到的文档中的 page_content 取出组合到一起
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt 模板
prompt = PromptTemplate(
    input_variables=["context", "history", "query"],
    template="""
你是一个专业的中文问答助手，擅长基于提供的资料回答问题。
请仅根据以下背景资料以及历史消息回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：{context}

历史消息：[{history}]

问题：{query}

回答：""",
)

# 大模型
llm = Tongyi(model="qwen-turbo", api_key=TONGYI_API_KEY)

# 历史消息
history = []

# 格式化历史消息
def format_history(history):
    # 只保留最近 3 轮对话记录
    max_epoch = 3
    if len(history) > 2 * max_epoch:
        history = history[-2 * max_epoch :]
    return "\n".join([f"{i['role']}：{i['content']}" for i in history])

# ！！！rephrase Prompt 模板
rephrase_prompt = PromptTemplate(
    input_variables=["history", "query"],
    template="""
根据历史消息简要完善用户的问题，使其更加具体。只输出完善后的问题。

历史消息：[{history}]

问题：{query}
""",
)

# ！！！重述链条：根据历史和当前 query 生成更具体问题
rephrase_chain = (
    {
        "history": lambda x: format_history(x["history"]),
        "query": lambda x: x["query"],
    }
    | rephrase_prompt
    | llm
    | StrOutputParser()
    | (lambda x: print(f"===== 重述后的查询: {x}=====") or x)
)

# Prompt 模板
prompt = PromptTemplate(
    input_variables=["context", "history", "query"],
    template="""
你是一个专业的中文问答助手，擅长基于提供的资料回答问题。
请仅根据以下背景资料以及历史消息回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：{context}

历史消息：[{history}]

问题：{query}

回答：""",
)

# ！！！RAG 链条:  使用重述后的 query进行检索
rag_chain = (
    {
        "context": lambda x: format_docs(
            retriever.invoke(
                rephrase_chain.invoke({"history": x.get("history"), "query": x.get("query")}),
                k=3,
            )
        ),
        "history": lambda x: format_history(x.get("history")),
        "query": lambda x: x.get("query"),
    }
    | prompt
    | (lambda x: print(x.text, end="") or x)
    | llm
    | StrOutputParser()  # 输出解析器，将输出解析为字符串
)

query_list = ["不动产或者动产被人占有怎么办", "那要是被损毁了呢"]
for query in query_list:
    print(f"===== 查询: {query} =====")
    response = rag_chain.invoke({"query": query, "history": history})
    print(response, end="\n\n")
    history.extend(
        [
            {"role": "用户", "content": query},
            {"role": "助手", "content": response},
        ]
    )


===== 查询: 不动产或者动产被人占有怎么办 =====
===== 重述后的查询: 不动产或者动产被人占有怎么办？如何依法维护自己的所有权？=====

你是一个专业的中文问答助手，擅长基于提供的资料回答问题。
请仅根据以下背景资料以及历史消息回答问题，如无法找到答案，请直接回答“我不知道”。

背景资料：第二分编　所有权

第四章　一般规定

第二百四十条　所有权人对自己的不动产或者动产，依法享有占有、使用、收益和处分的权利。

第二百四十一条　所有权人有权在自己的不动产或者动产上设立用益物权和担保物权。用益物权人、担保物权人行使权利，不得损害所有权人的权益。

第二百四十二条　法律规定专属于国家所有的不动产和动产，任何组织或者个人不能取得所有权。

第二百四十三条　为了公共利益的需要，依照法律规定的权限和程序可以征收集体所有的土地和组织、个人的房屋以及其他不动产。

征收集体所有的土地，应当依法及时足额支付土地补偿费、安置补助费以及农村村民住宅、其他地上附着物和青苗等的补偿费用，并安排被征地农民的社会保障费用，保障被征地农民的生活，维护被征地农民的合法权益。

征收组织、个人的房屋以及其他不动产，应当依法给予征收补偿，维护被征收人的合法权益；征收个人住宅的，还应当保障被征收人的居住条件。

任何组织或者个人不得贪污、挪用、私分、截留、拖欠征收补偿费等费用。

第五分编　占有

第二十章　占有

第四百五十八条　基于合同关系等产生的占有，有关不动产或者动产的使用、收益、违约责任等，按照合同约定；合同没有约定或者约定不明确的，依照有关法律规定。

第四百五十九条　占有人因使用占有的不动产或者动产，致使该不动产或者动产受到损害的，恶意占有人应当承担赔偿责任。

第四百六十条　不动产或者动产被占有人占有的，权利人可以请求返还原物及其孳息；但是，应当支付善意占有人因维护该不动产或者动产支出的必要费用。

第四百六十一条　占有的不动产或者动产毁损、灭失，该不动产或者动产的权利人请求赔偿的，占有人应当将因毁损、灭失取得的保险金、赔偿金或者补偿金等返还给权利人；权利人的损害未得到足够弥补的，恶意占有人还应当赔偿损失。

第四百六十二条　占有的不动产或者动产被侵占的，占有人有权请求返还原物；对妨害占有的行为，占有人有权请求排除妨害或者消除危险；因侵占或者妨害造成损害的，占有